In [3]:
import operator
import typing as tp

In [4]:
ParserP = tp.Callable[[str], tp.Tuple[tp.Any, str]]
# A simple parser
example_parser = lambda s: (s[0], s[1:])

In [5]:
class ParserError(Exception):
    def __init__(self, msg, content):
        super().__init__(f"{msg}: {content}")

def parse(p: ParserP, s:str) -> tp.Tuple[tp.Any, str]:
    (a, s) = p(s)
    return (a, s)

def anyChar() -> ParserP:
    def func(s):
        return (s[0], s[1:])
    return func

In [6]:
parse(anyChar(), "hello world")

('h', 'ello world')

In [8]:
def oneChar(c) -> ParserP:
    def func(s):
        if s[0] == c:
            return (s[0], s[1:])
        raise ParserError(f"Unexpected {s[0]}, expecting {c}", s)

    return func

In [9]:
parse(oneChar('h'), 'hello world')

('h', 'ello world')

In [10]:
def anyDigit() -> ParserP:
    def func(s):
        if s[0].isdigit():
            return (s[0], s[1:])
        raise ParserError(f"Expected digit, got {s[0]}", s)

    return func

In [11]:
parse(anyDigit(), '123')

('1', '23')

In [12]:
# Generic Predicate Parser
def satisfy(pred_function: tp.Callable[["char"], bool]) -> ParserP:
    def func(s):
        if not s:
            raise ParserError("Empty string", "")
        if pred_function(s[0]):
            return (s[0], s[1:])
        raise ParserError(f"Unexpected condition", s)
    
    return func

In [13]:
def oneCharP(c) -> ParserP:
    return satisfy(lambda c1: c == c1)

def anyDigitP() -> ParserP:
    return satisfy(lambda c: c.isdigit())

In [15]:
def compose(p1: ParserP, p2: ParserP) -> ParserP:
    def func(s):
        (a, s1) = parse(p1, s)
        (b, s2) = parse(p2, s1)
        return ((a, b), s2)
    
    return func

In [18]:
hp = oneChar('h')
ep = oneChar('e')
parse(compose(hp, ep), "hello world")

(('h', 'e'), 'llo world')

In [19]:
def choice(p1: ParserP, p2: ParserP) -> ParserP:
    def func(s):
        try:
            return p1(s)
        except ParserError:
            return p2(s)
    
    return func

In [20]:
def mathOp(op):
    return satisfy(lambda c: c == op)

def mathOpP() -> ParserP:
    plus = mathOp("+")
    minus = mathOp("-")
    mult = mathOp("*")
    div = mathOp("/")

    return choice(plus, choice(minus, choice(mult, minus)))

In [21]:
def expression_does_not_work():
    def func(s):

        (digit1, s1) = parse(anyDigitP(), s)
        (op, s2) = parse(mathOpP(), s1)
        (digit2, s3) = parse(anyDigitP(), s2) # this does not work

        return ((int(digit1), op, int(digit2)), s3)

    return func

In [22]:
parse(expression_does_not_work(), "1+2")

((1, '+', 2), '')